# <center> **Coded Apertures: 2D Image Recostruction Test 2**<center>

**Libraries**

In [1]:
import collections.abc as c

import numpy as np
from sympy import nextprime
from matplotlib.colors import ListedColormap
from scipy.signal import correlate

import plot_module as plot
import test_reconstruction as test

np.set_printoptions(precision=2, suppress=False)

In [ ]:
import importlib
importlib.reload(plot)
importlib.reload(test)

## **2D URA/MURA Masks**

**URA/MURA Pattern Simulation**

**Mask Interface**

**Sky Simulation**

**Sky Reconstruction Simulation**

**Tests**

### **URA Mask Pattern**

### **MURA Mask Pattern**